# **Aula 04 - Autenticação com MySQL** 🚀

Aprenda a implementar autenticação segura em aplicações **Node.js** utilizando **MySQL**.

✅ **Configuração do MySQL no Node.js** (usando `mysql2`)  
✅ **Criação de um sistema de login e registro**  
✅ **Hash de senhas** com `bcrypt` para garantir segurança  
✅ **Autenticação JWT** (`jsonwebtoken`) para gerar e validar tokens  
✅ **Middleware de autenticação** para proteger rotas e controlar acessos  

---

## **1️⃣ Configuração do MySQL no Node.js**

### **Instalação das dependências**
Primeiro, instale os pacotes necessários:

```sh
npm init -y
npm install express mysql2 dotenv bcrypt jsonwebtoken cors


# **📌 Comandos Básicos de MySQL para Autenticação**

Nesta aula, utilizamos **MySQL** para armazenar e gerenciar os usuários da aplicação.  
Aqui estão os comandos essenciais que você precisa entender para criar e manipular o banco de dados.  

---

## **1️⃣ Criando um Banco de Dados**
Antes de tudo, criamos um banco de dados chamado **`autenticacao`**:

```sql
CREATE DATABASE autenticacao;

Para usar esse banco de dados:


USE autenticacao;

# **2️⃣ Criando uma Tabela para Usuários**

Agora, criamos a tabela usuarios, que armazenará os dados dos usuários:

```sql

CREATE TABLE usuarios (
  id INT AUTO_INCREMENT PRIMARY KEY,
  nome VARCHAR(100) NOT NULL,
  email VARCHAR(100) UNIQUE NOT NULL,
  senha VARCHAR(255) NOT NULL
);
Explicação:

id INT AUTO_INCREMENT PRIMARY KEY → ID único para cada usuário.
nome VARCHAR(100) NOT NULL → Nome do usuário (até 100 caracteres, obrigatório).
email VARCHAR(100) UNIQUE NOT NULL → Email único e obrigatório.
senha VARCHAR(255) NOT NULL → Senha criptografada.

# **3️⃣ Inserindo Dados na Tabela**
Para cadastrar um usuário diretamente pelo MySQL:

```sql
INSERT INTO usuarios (nome, email, senha) 
VALUES ('João Silva', 'joao@email.com', 'senha_hash_criptografada');

# **4️⃣ Consultando Usuários**
Para visualizar todos os usuários cadastrados:

```sql
SELECT * FROM usuarios;
Se quiser buscar um usuário específico pelo email:

SELECT * FROM usuarios WHERE email = 'joao@email.com';

# **5️⃣ Atualizando Dados**
Para modificar o nome de um usuário:

```sql
UPDATE usuarios SET nome = 'João Pedro Silva' WHERE email = 'joao@email.com';

# **6️⃣ Deletando um Usuário**
Para excluir um usuário do banco de dados:

```sql
DELETE FROM usuarios WHERE email = 'joao@email.com';

# **SOBRE CONECTAR O DB AO CODIGO**


---

Crie um arquivo chamado `db.js` para gerenciar a conexão com o MySQL:

```javascript
require('dotenv').config();
const mysql = require('mysql2');

const connection = mysql.createConnection({
  host: process.env.DB_HOST,
  user: process.env.DB_USER,
  password: process.env.DB_PASSWORD,
  database: process.env.DB_DATABASE
});

connection.connect(err => {
  if (err) {
    console.error('🚫 Erro ao conectar ao MySQL:', err);
    return;
  }
  console.log('✅ Conectado ao MySQL!');
});

module.exports = connection;


# 🚀 **Introdução ao bcrypt e JWT** 🔐

Nesta seção, vamos conhecer dois conceitos essenciais para a segurança em aplicações web: **bcrypt** e **JWT (JSON Web Token)**. Estes serão seus primeiros passos na construção de um sistema de autenticação seguro! 😃

---

## 🔒 **bcrypt**

**bcrypt** é uma biblioteca usada para **hashing** de senhas, ou seja, transformar uma senha simples em uma sequência de caracteres embaralhada, chamada **hash**. Esse processo é **unidirecional** (não é possível reverter o hash para descobrir a senha original), garantindo assim a proteção das senhas dos usuários, mesmo se o banco de dados for comprometido. 💪

### Por que utilizar bcrypt?  
- **Segurança:** As senhas não são armazenadas em texto simples. 🔑  
- **Proteção contra ataques:** Invasores terão dificuldade em recuperar as senhas originais. 🚫  
- **Facilidade de uso:** Funções para gerar e comparar hashes simplificam a implementação da segurança. 👍

### Exemplo de uso do bcrypt:
```javascript
const bcrypt = require('bcrypt');

// Função para gerar o hash de uma senha
async function gerarHash(senha) {
  const saltRounds = 10; // Define a complexidade do hash
  const hash = await bcrypt.hash(senha, saltRounds);
  return hash;
}

// Função para comparar uma senha com o hash armazenado
async function compararSenha(senha, hashArmazenado) {
  const resultado = await bcrypt.compare(senha, hashArmazenado);
  return resultado; // Retorna true se as senhas coincidirem
}

// Exemplo de uso:
(async () => {
  const senha = 'minhaSenha123';
  const hashGerado = await gerarHash(senha);
  console.log('🔐 Hash gerado:', hashGerado);

  const senhaValida = await compararSenha('minhaSenha123', hashGerado);
  console.log('✅ Senha válida?', senhaValida);
})();


# **🌟 JWT (JSON Web Token)**

JWT é um padrão aberto para criar tokens que garantem a autenticação e autorização dos usuários. Um token JWT é composto por três partes: header, payload e signature. O payload contém informações úteis (como o ID do usuário e o e-mail) e é assinado digitalmente para garantir que não foi alterado. 🔏

**Por que utilizar JWT?**
Autenticação sem estado: Permite que a API valide usuários sem precisar manter sessões no servidor. 🖥️
Transmissão segura de informações: Os tokens podem ser verificados para garantir que os dados não foram modificados. 🔍
Flexibilidade: O JWT pode ser usado para autorizar o acesso a diferentes partes da aplicação. 🚀

exemplo de uso do JWT:
```javascript
const jwt = require('jsonwebtoken');

const secretKey = 'seu_segredo_jwt'; // Chave secreta para assinar o token

// Função para gerar um token JWT
function gerarToken(dadosUsuario) {
  // dadosUsuario é um objeto com informações do usuário, por exemplo: { id: 1, email: 'user@example.com' }
  const token = jwt.sign(dadosUsuario, secretKey, { expiresIn: '1h' });
  return token;
}

// Função para verificar e decodificar um token JWT
function verificarToken(token) {
  try {
    const decoded = jwt.verify(token, secretKey);
    return decoded;
  } catch (err) {
    // Caso o token seja inválido ou esteja expirado
    return null;
  }
}

// Exemplo de uso:
const usuario = { id: 1, email: 'user@example.com' };
const tokenGerado = gerarToken(usuario);
console.log('🚀 Token gerado:', tokenGerado);

const dadosDecodificados = verificarToken(tokenGerado);
console.log('🔍 Dados decodificados do token:', dadosDecodificados);

# **📚 Resumo**

**bcrypt:**

O que é: Uma biblioteca para hash de senhas. 🔒
Por que usar: Transforma senhas em um formato seguro e irreversível. 🔑
Como usar: Gera um hash da senha e compara senhas fornecidas com o hash armazenado. ✅

**JWT (JSON Web Token):**

O que é: Um padrão para criar tokens de autenticação que transportam informações do usuário de forma segura. 🔐
Por que usar: Permite autenticar e autorizar usuários sem manter sessões no servidor, facilitando a escalabilidade. 🌐
Como usar: Gera tokens com dados do usuário e os verifica em cada requisição para controlar o acesso. 🚦

# 🚀 Exercícios de Rotas de Autenticação e Autorização

Estes exercícios têm o objetivo de aplicar os conceitos de autenticação e autorização usando rotas em uma aplicação Node.js. Vocês deverão chegar às respostas com base nas informações fornecidas em aula, sem dicas diretas de código. Desafiem-se a pensar e implementar as soluções!

---

## Exercício 1: Rota de Cadastro de Usuário 📝

**Objetivo:**  
Desenvolver uma rota para cadastro de usuários, onde os dados a serem registrados são:  
- **id_usuario**  
- **nome_usuario**  
- **email_usuario**  
- **data_cadastro**  
- **tipo_usuario**  
- **senha** (A senha deve ser armazenada de forma segura, ou seja, **criptografada**).

**Desafio:**  
- Planeje a estrutura da rota de cadastro.  
- Garanta que, antes de salvar os dados, a senha seja devidamente criptografada.  
- Pense em como validar as informações recebidas e responder adequadamente em caso de erros.

---

## Exercício 2: Rota de Login e Geração de Token 🔐

**Objetivo:**  
Criar uma rota de login onde o usuário informe seu **email** e **senha** já cadastrados. Ao validar essas credenciais, um **token de acesso** deverá ser gerado e retornado.

**Desafio:**  
- Verifique se o email e a senha correspondem aos dados cadastrados.  
- Gere um token de acesso para o usuário autenticado.  
- Assegure que o token seja retornado de forma segura para que o usuário possa usá-lo em requisições subsequentes.

---

## Exercício 3: Rota para Retornar Todos os Usuários (Usuários Autenticados) 👥

**Objetivo:**  
Implementar uma rota que retorne uma lista de usuários com os seguintes campos: **nome**, **email** e **tipo_usuario**.  
Esta rota deverá ser acessível somente se o **token de acesso** fornecido for válido.

**Desafio:**  
- Faça a verificação da validade do token antes de processar a requisição.  
- Se o token for válido, retorne os dados dos usuários conforme solicitado.  
- Caso o token seja inválido ou ausente, negue o acesso e retorne uma mensagem de erro.

---

## Exercício 4: Rota Exclusiva para Administradores 🔥

**Objetivo:**  
Criar uma rota que permita buscar todos os usuários, retornando os campos: **nome**, **email**, **tipo_usuario** e **data_cadastro**.  
Esta rota deve ser restrita apenas a usuários do tipo **admin** e só pode ser acessada se o **token de acesso** for válido.

**Desafio:**  
- Verifique a validade do token de acesso para qualquer requisição.  
- Adicione uma camada extra de autorização para garantir que somente usuários com o tipo **admin** possam acessar esta rota.  
- Caso o usuário não seja um admin ou o token seja inválido, negue o acesso e forneça uma mensagem de erro apropriada.

---

# 💡 Dicas Gerais

- **Revisem os conceitos:** Lembrem-se do que foi discutido sobre criptografia de senhas, geração de tokens e proteção de rotas.  
- **Planejamento:** Antes de codificar, façam um esboço da lógica de cada rota, pensando em como será a verificação de credenciais e autorização.  
- **Validação:** Sempre valide as entradas do usuário e trate os casos em que os dados estejam incorretos ou o token não seja válido.  
- **Discussão:** Troquem ideias com os colegas e discutam possíveis abordagens para resolver os desafios propostos.

Boa prática e mãos à obra! 🚀😃
